# Obtain weekday prototypes
The main objective of this task is to create every weekday prototypes. We want to model two types of days based on the consumption activity of each building:
- **Active** day.
- **Inactive** day.

Thus, for each counter on the database, we'll get 13 day prototypes (6 working days * 2 types of days + 1 inactive day corresponding to Sundays). Moreover, there are 97 different buildings, so we expect to get 13 * 97 prototypical days.

This previously mentioned activity will be defined by calculating the mean of Sundays' consumptions for each building (greater than this value plus some margin will indicate an active day; lower or equal than this value plus the margin should be labelled as an inactive day).

#### Directory structure
./<br></br>
notebook/<br></br>
    &emsp;|--- data_preprocessing<br></br>
    &emsp;&emsp;&emsp;&emsp;|--- weekday_prototypes.ipynb<br></br>
out/<br></br>
    &emsp;|--- consumptions.zip

In [1]:
CONS_PATH = 'C:/Users/thmas/OneDrive - Universidad de Castilla-La Mancha/Informática/TFG/out/'

In [2]:
import pandas as pd
import numpy as np

In [3]:
raw = pd.read_pickle(CONS_PATH + 'consumptions.zip')
raw

,building_id,weekday,active,type,consumptions
day,,,,,
2011-07-26,27,1,False,1,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
2011-07-27,27,2,True,2,"[17.0, 19.0, 18.35079465, 35.84631282, 47.8462..."
2011-07-28,27,3,True,2,"[18.88870418, 18.80300889, 18.84589226, 35.845..."
2011-07-29,27,4,True,2,"[20.0, 21.0, 20.0, 37.78877899, 45.84570469, 5..."
2011-07-30,27,5,False,1,"[17.29811321, 17.0, 17.23969745, 17.84583302, ..."
...,...,...,...,...,...
2020-05-10,2233,6,False,1,"[8.69106633, 8.54560612, 8.13895298, 8.8050705..."
2020-05-11,2233,0,False,1,"[8.57166388, 10.30621946, 10.4596744, 12.97239..."
2020-05-12,2233,1,False,1,"[8.31569322, 9.21411531, 10.19029213, 12.96810..."


In [4]:
def get_prototype(df: pd.DataFrame, mode: str = 'mean') -> pd.DataFrame:
    weekday = df['weekday'].iloc[0]
    active = df['active'].iloc[0]
    consumer_type = df['type'].iloc[0]
    
    cons = []
    for i in range(24):
        i_consumptions = []
        for j in range(df.shape[0]):
            i_consumptions.append(df['consumptions'].iloc[j][i])

        if mode == 'std':
            cons.append(np.nanstd(i_consumptions))
        else:
            cons.append(np.nanmean(i_consumptions))
            
    return pd.DataFrame({'weekday': weekday, 'active': active, 'type': consumer_type, 'consumptions': [cons]})

In [5]:
mean_proto, std_proto = pd.DataFrame(), pd.DataFrame()
for d in range(0, 7):
    df = raw[raw['weekday'] == d]
    
    for a in (True, False):
        df_a = df[df['active'] == a]
        
        for t in df_a['type'].unique():
            df_t = df_a[df_a['type'] == t]
            
            mean_proto = mean_proto.append(get_prototype(df_t, mode='mean'), ignore_index=True)
            std_proto = std_proto.append(get_prototype(df_t, mode='std'), ignore_index=True)

mean_proto

,weekday,active,type,consumptions
0,0,True,2,"[40.146782594365376, 51.88675704337931, 68.832..."
1,0,True,1,"[10.847154258209107, 14.227894442444653, 17.31..."
2,0,True,0,"[2.0464377459317045, 2.4123846483826723, 2.780..."
3,0,False,1,"[19.686508066069237, 22.02506422218569, 24.189..."
4,0,False,0,"[1.9675573138662967, 2.390020343727421, 2.7358..."
5,1,True,2,"[40.65917479534745, 52.51305171432206, 69.8959..."
6,1,True,1,"[10.95807251304097, 14.245881103643974, 17.405..."
7,1,True,0,"[2.1638190668084425, 2.516558698722412, 2.8718..."
8,1,False,1,"[19.91485033133719, 22.394262673761535, 24.844..."
9,1,False,0,"[2.082762755846858, 2.501112811568701, 2.91188..."
